In [23]:
from random import random
from math import log
import numpy as np
from typing import Callable 
from numpy.typing import ArrayLike
from scipy.stats import expon

---
# Ejercicio 7
Generar los valores correspondientes a 30 variables aleatorias exponenciales independientes, cada una con media 1. Luego, en base al estadístico de prueba de Kolmogorov-Smirnov, aproxime el $p_{valor}$ de la prueba de que los datos realmente provienen de una distribución exponencial con media 1.


In [ ]:
# Variable aleatoria exponenial
def XEXP():
    U = 1 - random()
    return -log(1 - U)

# Generamos valores correspondientes a 30 exponenciales independientes
samples = [XEXP() for _ in range(30)]
#Calculamos la cantidad de muestras -> 30 :v
NSAMPLES = len(samples)

In [31]:
#Definimos el estadístico
def KS_statistic(Nsamples:int, samples:ArrayLike, G:Callable[[ArrayLike], ArrayLike]) -> float:
    Fe_samples = np.arange(1, Nsamples + 1, 1) / Nsamples
    G_values = G(samples)

    Fe_minus = np.arange(0, Nsamples) / Nsamples
    Fe_plus = np.arange(1, Nsamples + 1) / Nsamples

    D_minus = G_values - Fe_minus
    D_plus = Fe_plus - G_values
    
    d = np.max(np.concatenate([D_plus, D_minus]))
    return d

#Test de Kolomogorov-Smirnov
def KS_test(Nsim:int, Nsamples:int,
            samples:ArrayLike, alpha:float,
            G:Callable[[ArrayLike], ArrayLike]):

    p_value = 0
    #Ordeño muestras 🐄
    samples = np.sort(samples)
    #Calculo estadístico inicial
    d0 = KS_statistic(Nsamples=Nsamples, samples=samples, G=G)

    for _ in range(Nsim):
        #Generamos muestras aleatorias para estimar el p_valor
        uniform_samples = np.sort([random() for _ in range(Nsamples)])
        d_sim = KS_statistic(Nsamples=Nsamples, samples=uniform_samples, G=G)
        if d_sim >= d0: 
            p_value += 1
        
    p_value = p_value / Nsim
    print(f"☝️ p-valor obtenido: {round(p_value, 4)}")

    if p_value > alpha:
        print(f"😲☝️ Como {p_value} > {alpha}:")
        print("\t 😒 No hay evidencia suficiente para rechazar Ho")
    else:
        print(f"😲☝️ Como {p_value} <= {alpha}:")
        print(f"\t 🔴 Se rechaza Ho con una confianza del {int(100 * (1-alpha))}%")

In [35]:
KS_test(Nsim=10_000, Nsamples=NSAMPLES, samples=samples, alpha=0.05, G=expon.cdf)

☝️ p-valor obtenido: 1.0
😲☝️ Como 1.0 > 0.05:
	 😒 No hay evidencia suficiente para rechazar Ho
